In [12]:
#coding=utf-8

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import ssl
from lxml import etree
import urllib
import time
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

chrome_driver = r'C:/Users/Ni/Desktop/ni/OS/chromedriver'
ssl._create_default_https_context = ssl._create_unverified_context
chrome_options = Options()

In [13]:
def ParseCookiestr(cookie_str):
	cookielist = []
	for item in cookie_str.split(';'):
		cookie={}
		itemname=item.split('=')[0]
		iremvalue=item.split('=')[1]
		cookie['name']=itemname
		cookie['value']=urllib.parse.unquote(iremvalue)
		cookielist.append(cookie)
	return cookielist

In [14]:
url = 'https://max.pedata.cn/client/org/list?menuItem=3'
# cookielist=ParseCookiestr("Hm_lvt_0d052dd2e4e34a214d13e86591990a09=1659661345; SESSION=ZjI2NjJiZmQtYjZkYi00OWJlLTlkYzYtOGM4ZWI2YTYxMDk1; Hm_lpvt_0d052dd2e4e34a214d13e86591990a09=1659702557")
browser = webdriver.Chrome(executable_path=chrome_driver, chrome_options=chrome_options)
browser.get(url)

C:\Users\Ni\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Ni\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
cookies = browser.get_cookies()
cookies

[{'domain': '.max.pedata.cn',
  'httpOnly': False,
  'name': 'Hm_lpvt_0d052dd2e4e34a214d13e86591990a09',
  'path': '/',
  'secure': False,
  'value': '1659924456'},
 {'domain': 'max.pedata.cn',
  'httpOnly': True,
  'name': 'SESSION',
  'path': '/',
  'sameSite': 'Lax',
  'secure': False,
  'value': 'ZmJkMzgxNTAtM2I1Yy00YmFlLWFhMTctNWUxNWY1YTI1MWZl'},
 {'domain': '.max.pedata.cn',
  'expiry': 1691460455,
  'httpOnly': False,
  'name': 'Hm_lvt_0d052dd2e4e34a214d13e86591990a09',
  'path': '/',
  'secure': False,
  'value': '1659924445'},
 {'domain': 'max.pedata.cn',
  'expiry': 1975284455,
  'httpOnly': False,
  'name': '_uab_collina',
  'path': '/client',
  'secure': False,
  'value': '165992445544953673243396'}]

In [16]:
for cookie in cookies:
    browser.add_cookie(cookie)
browser.get(url)

In [389]:
jg_urls = []

In [397]:
for i in range(1):
    time.sleep(np.random.rand() + 1.7)
    r = browser.page_source
    h = etree.HTML(r)
    jg_pagei_urls = h.xpath('/html//div[2]/div[3]/div[1]/div/div/div/div/div[2]//td[1]//div[2]/a/@href')
    jg_urls = jg_urls + jg_pagei_urls
    ac = ActionChains(browser)
    ac.move_to_element(browser.find_element_by_xpath("/html/body/div/div/div/div[2]/div[3]/div[2]/ul/li[@title='下一页']")).perform()
    ac.click(browser.find_element_by_xpath("/html/body/div/div/div/div[2]/div[3]/div[2]/ul/li[@title='下一页']")).perform()
jg_urls = list(set(jg_urls))
len(jg_urls)

28487

In [398]:
with open('./jg_urls.txt', 'w') as f:
    f.write('\n'.join(jg_urls))

In [17]:
with open('./jg_urls.txt', 'r') as f:
    jg_urls = f.read().split()

In [32]:
from sqlalchemy import create_engine

class MySQLInsert:
    def __init__(self, table):
        self.host = "localhost"
        self.port = "3306"
        self.username = "root"
        self.password = "Amous0908"
        self.db = "db01"
        self.table = table
        self.write_mode = "append"
        self.connect_url = "mysql+pymysql://" + self.username + ':' + self.password + '@' + self.host + ':' + self.port + '/' + self.db + '?charset=utf8'
        self.mysql_connect = create_engine(self.connect_url)

    def df_write_mysql(self, df):
        pd.io.sql.to_sql(df, self.table, self.mysql_connect, schema=self.db, if_exists=self.write_mode)


In [ ]:
from collections import defaultdict
import pandas as pd
from datetime import datetime
from selenium.webdriver.common.by import By
import re
from tqdm import tqdm

mysqlinsert_basic_info = MySQLInsert('机构基本信息')
mysqlinsert_fund_manager = MySQLInsert('基金管理人')
mysqlinsert_funds = MySQLInsert('直投基金')

def unit_replace(s):
    s = s.replace(' ', '')
    s = s.replace('人民币', '')
    s = s.replace('美元', '*6.7622')
    s = s.replace('万', '*10000')
    s = s.replace('亿', '*100000000')
    t = s.split('*')
    r = 1.0
    try:
        for i in t:
            r *= float(i)
    except:
        return 0.0
    return r / 10000

def datestr_process(s):
    s = s.replace(' ', '')
    if s == '--':
        s = '1900-01-01'
    return datetime.strptime(s, '%Y-%m-%d')
    
flag1 = 0
flag2 = 1
flag3 = 1
start = 6854
flag1 += start
flag2 += 2172
flag3 += 12710
for jg_url_part in tqdm(jg_urls[start:], position=0):
    flag1 += 1
    data = defaultdict(list)
    jg_url = 'https://max.pedata.cn' + jg_url_part
    browser.get(jg_url)
    time.sleep(np.random.rand() + 2.4)
    r = browser.page_source
    h = etree.HTML(r)
    
    # ******************机构基本信息******************
    # 机构名称
    name = h.xpath('/html/body/div/div/div/div/div/div[1]/div[2]/div[1]/text()')
#     print('='*50)
#     print('{}/{}'.format(flag1, len(jg_urls)+1).center(50, '='))
#     print('{}'.format(name[0]).center(50, '*'))
#     print('-'*50)
#     print('{}'.format('1. 机构信息').center(50, '*'))
    
    # 标签
    tags = h.xpath('/html/body/div/div/div/div/div/div[1]/div[2]/div[2]/span/a/text()')
    # 成立时间
    found_time = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[2]/span[2]/text()')
#     if found_time[0].replace(' ', '') == '--':
#         found_time = '1900-01-01'
#     else:
#         found_time = found_time[0].replace(' ', '')
#     found_time = datetime.strptime(found_time, '%Y-%m-%d')
    found_time = datestr_process(found_time[0])
    # 管理资本量
    capital = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[3]/span[2]/text()')
    capital = unit_replace(capital[0])
    # 总部办公地
    location = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[5]/span[2]/text()')
    # 管理基金数
    funds_num = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[7]/span[2]/text()')
    if funds_num[0].replace(' ', '') == '--':
        funds_num = 0
    else:
        funds_num = int(funds_num[0].replace(' ', '').replace('支', ''))
    # 最新投资时间
    lastest_investment_time = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[6]/span[2]/text()')
#     if lastest_investment_time[0].replace(' ', '') == '--':
#         lastest_investment_time = '1900-01-01'
#     else:
#         lastest_investment_time = lastest_investment_time[0].replace(' ', '')
#     lastest_investment_time = datetime.strptime(lastest_investment_time, '%Y-%m-%d')
    lastest_investment_time = datestr_process(lastest_investment_time[0])
    # 投资项目数
    projects_num = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[8]/span[2]/text()')
    if projects_num[0].replace(' ', '') == '--':
        projects_num = 0
    else:
        projects_num = int(projects_num[0].replace(' ', '').replace('个', ''))
    # 机构类型
    classify = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[10]/span[2]/text()')
    # 单项投资金额最小值与最大值
    investment_amount_per_project_list = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[11]/span[2]/text()')
    if len(investment_amount_per_project_list) != 0:
        investment_amount_per_project = investment_amount_per_project_list[0].replace(' ', '')
        if '~' in investment_amount_per_project:
            investment_amount_min_max = investment_amount_per_project.split('~')
            investment_amount_min = investment_amount_min_max[0]
            investment_amount_max = investment_amount_min_max[-1]
        elif investment_amount_per_project == '--':
            investment_amount_min = '0'
            investment_amount_max = '0'
        else:
            investment_amount_min = investment_amount_per_project
            investment_amount_max = investment_amount_per_project
        investment_amount_min = unit_replace(investment_amount_min)
        investment_amount_max = unit_replace(investment_amount_max)
    else:
        investment_amount_max, investment_amount_min = 0, 0    
    # 投资阶段
    investment_stage = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[12]/span[2]/div/text()')
    # 主投行业
    industry = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[13]/span[2]/text()')
    # 简介
    summary = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[1]/div[2]/ul/li[14]/span[2]/text()')
    # 电话
    tel = h.xpath('/html/body/div/div/div/div/div/div[1]/div[2]/div[3]/div[1]/span[2]/text()')
    
    data['名称'].append(name[0].replace(' ', ''))
    data['标签'].append(','.join(tags))
    data['成立时间'].append(found_time)
    data['管理资本量（万元）'].append(capital)
    data['总部办公地'].append(','.join(location))
    data['管理基金数'].append(funds_num)
    data['最新投资时间'].append(lastest_investment_time)
    data['投资项目个数'].append(projects_num)
    data['机构类型'].append(','.join(classify))
    data['单项投资金额最小值（万元）'].append(investment_amount_min)
    data['单项投资金额最大值（万元）'].append(investment_amount_max)
    data['投资阶段'].append(','.join(investment_stage))
    data['主投行业'].append(','.join(industry))
    data['简介'].append(','.join(summary))
    data['电话'].append('--' if len(''.join(re.findall("\d+", ''.join(tel)))) < 8 else ''.join(re.findall("\d+", ''.join(tel))))
    tdf = pd.DataFrame.from_dict(dict(data))
    tdf.index = pd.Series([flag1])
#     print(tdf)
    mysqlinsert_basic_info.df_write_mysql(tdf)
    
    # ****************基金管理人信息*******************
    ac = ActionChains(browser)
#     ac.move_to_element(browser.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
#     ac.click(browser.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
    ac.move_to_element(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
    ac.click(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
    time.sleep(1.2)
    r = browser.page_source
    h = etree.HTML(r)
    
    pages = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[2]/ul/li[last()-2]/a/text()")
    if len(pages) != 0:
        pages = int(pages[0])
    else:
        pages = 1
#     print('-'*50)
#     print('{}'.format('2. 基金管理人信息').center(50, '*'))
#     print('共{}页'.format(pages).center(50, '*'))
    
    data = defaultdict(list)
    t_flag2 = flag2
    while pages > 0:
        fund_managers = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[2]/div/div[2]/table/tbody/tr/td[1]/div/span/a/text()")
        fund_managers_num = len(fund_managers)
        t_flag2 += fund_managers_num
        fund_managers_found_time = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[2]/div/text()")
        fund_managers_code = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody//td[3]/div/text()')
        fund_managers_start_time = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody//td[4]/div/text()")
        fund_managers_rep = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[5]/div/div/text()')
        fund_managers_capital = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[7]/div/text()")
        fund_managers_i_o_ratio = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[8]/div/text()")

        for i in range(fund_managers_num):
            data['名称'].append(name[0].replace(' ', ''))
            data['基金管理人'].append(fund_managers[i])
            data['基金管理人备案时间'].append(datestr_process(fund_managers_found_time[i]))
            data['基金管理人备案编号'].append(fund_managers_code[i].replace(' ', ''))
            data['基金管理人成立时间'].append(datestr_process(fund_managers_start_time[i]))
            data['基金管理人法人代表'].append(fund_managers_rep[i].replace(' ', ''))
            fund_managers_fund = h.xpath('/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr[' + str(i+1) + ']/td[6]/div/div/div/div/div/a/text()')
            data['基金管理人管理基金'].append(','.join(fund_managers_fund).replace(' ', ''))
            data['基金管理人管理资本量（万元）'].append(unit_replace(fund_managers_capital[i]))
            data['基金管理人投资/退出数量'].append(fund_managers_i_o_ratio[i].replace(' ', ''))
        pages -= 1
        if pages > 0:
            ac.move_to_element(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[2]/ul/li[last()-1]/a")).perform()
            ac.click(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]/div[2]/ul/li[last()-1]/a")).perform()
            time.sleep(0.8)
            
    tdf = pd.DataFrame.from_dict(dict(data))
    tdf.index = pd.Series(list(range(flag2, t_flag2)))
    mysqlinsert_fund_manager.df_write_mysql(tdf)
    flag2 = t_flag2
    
    # ***************直投基金信息***************
#     ac.move_to_element(browser.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
#     ac.click(browser.find_element_by_xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
    try:
        ac.move_to_element(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
        ac.click(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
    except:
        ac.move_to_element(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
        ac.click(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[1]/div/div/div/div/div[1]/div[2]/a")).perform()
        
    time.sleep(1.4)
    r = browser.page_source
    h = etree.HTML(r)
    
    pages = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[last()-2]/a/text()")
    if len(pages) != 0:
        pages = int(pages[0])
    else:
        pages = 1
        
#     print('-'*50)
#     print('{}'.format('3. 直投基金信息').center(50, '*'))
#     print('共{}页'.format(pages).center(50, '*'))
    
    data = defaultdict(list)
    t_flag3 = flag3
    while pages > 0:
        funds = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[2]/div/div[2]/table/tbody/tr/td[1]/div/span/a/text()")
        funds_num = len(funds)
        t_flag3 += funds_num
        funds_scale = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[2]/div/text()")
        funds_found_time = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[3]/div/text()")
        funds_code = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[4]/div/text()")
        funds_start_time = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[5]/div/text()")
        funds_manager = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[6]/div/a/text()")
        funds_investment_num = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[7]/div/text()")
        funds_lastest_investment_time = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[8]/div/text()")
        funds_lastest_investment_project = h.xpath("/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/div[2]/table/tbody/tr/td[9]/div//text()")
        for i in range(funds_num):
            data['名称'].append(name[0].replace(' ', ''))
            data['基金名称'].append(funds[i].replace(' ', ''))
            data['目标规模（万元）'].append(unit_replace(funds_scale[i].replace(' ', '')))
            data['备案时间'].append(datestr_process(funds_found_time[i]))
            data['备案编号'].append(funds_code[i].replace(' ', ''))
            data['成立时间'].append(datestr_process(funds_start_time[i]))
            data['基金管理人'].append(funds_manager[i].replace(' ', ''))
            data['投资数量'].append(0 if funds_investment_num[i].replace(' ', '') == '--' else int(funds_investment_num[i]))
            data['最新投资时间'].append(datestr_process(funds_lastest_investment_time[i]))
            data['最新投资'].append(funds_lastest_investment_project[i].replace(' ', ''))
        pages -= 1
        if pages > 0:
            ac.move_to_element(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[last()-1]/a")).perform()
            ac.click(browser.find_element(By.XPATH, "/html/body/div/div/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/div[2]/div[2]/ul/li[last()-1]/a")).perform()
            time.sleep(1.4)
    tdf = pd.DataFrame.from_dict(dict(data))
    tdf.index = pd.Series(list(range(flag3, t_flag3)))
    mysqlinsert_funds.df_write_mysql(tdf)
    flag3 = t_flag3

  1%|█                                                                          | 304/21633 [37:23<42:38:55,  7.20s/it]